<a href="https://colab.research.google.com/github/zeylulbalci/dsa210project/blob/main/Full_TMDB_Movies_Dataset_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

# Step 1: Load your main dataset
df_main = pd.read_csv("IMDb___Gross_Dataset.csv")

# Clean titles and ensure year is numeric
df_main['clean_title'] = df_main['Movie'].astype(str).str.lower().str.replace(r'[^\w\s]', '', regex=True).str.strip()
df_main['Year'] = pd.to_numeric(df_main['Year'], errors='coerce')

# Step 2: Define columns to keep from TMDB dataset
columns_to_keep = ['title', 'release_date', 'budget', 'revenue', 'runtime', 'genres', 'popularity', 'vote_average', 'vote_count']
chunksize = 100_000
matched_rows = []

# Step 3: Read TMDB in chunks (safe read with error skipping)
for chunk in pd.read_csv(
    "TMDB_movie_dataset_v11.csv",
    usecols=columns_to_keep,
    chunksize=chunksize,
    encoding='utf-8',
    on_bad_lines='skip',
    low_memory=False
):
    chunk = chunk.copy()  # Avoid SettingWithCopyWarning

    # Preprocess TMDB chunk
    chunk['clean_title'] = chunk['title'].astype(str).str.lower().str.replace(r'[^\w\s]', '', regex=True).str.strip()
    chunk['release_date'] = pd.to_datetime(chunk['release_date'], errors='coerce')
    chunk = chunk.dropna(subset=['release_date'])
    chunk['release_year'] = chunk['release_date'].dt.year.astype(int)

    # Merge on clean_title + year
    merged = chunk.merge(df_main, how='inner', left_on=['clean_title', 'release_year'], right_on=['clean_title', 'Year'])
    matched_rows.append(merged)

# Step 4: Concatenate all matched chunks
df_matched = pd.concat(matched_rows, ignore_index=True)

# Step 5: Select and rename relevant columns
df_final = df_matched[[
    'Movie', 'Year', 'budget', 'revenue', 'runtime', 'genres', 'popularity',
    'vote_average', 'vote_count', 'Views', 'Likes', 'Comments',
    'IMDb_Rating', 'numVotes', 'Gross'
]]

# Step 6: Save final enriched dataset
df_final.to_csv("Enriched_Movie_Dataset.csv", index=False)
print("✅ Done! Enriched dataset created with", df_final.shape[0], "matched rows.")


<ipython-input-1-4da5be5e1d7b>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['release_year'] = chunk['release_date'].dt.year.astype(int)
<ipython-input-1-4da5be5e1d7b>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['release_year'] = chunk['release_date'].dt.year.astype(int)
<ipython-input-1-4da5be5e1d7b>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

✅ Done! Enriched dataset created with 280 matched rows.
